In [1]:
import docx
import os 
import collections
import subprocess
from docx.shared import RGBColor
import nltk
import sklearn
import re
import string
import os
import urllib
import itertools
import tarfile
from nltk.corpus import stopwords
import pymorphy2
import pickle
import re, string, timeit
from sklearn.feature_extraction.text import TfidfVectorizer
morph = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words("russian")
russian_stopwords.remove('можно')
russian_stopwords.remove('быть')
russian_stopwords.remove('или')
russian_stopwords.remove('был')
russian_stopwords.remove('может')
russian_stopwords.remove('нельзя')
russian_stopwords.remove('всегда')
russian_stopwords.remove('более')
russian_stopwords.remove('без')

In [2]:
highlights_red = collections.defaultdict(list)
problem_red = []
for region in os.listdir('DataSet_Razmetra'):
    if region == 'РСО-Алания':
        continue
    for corupt_type in os.listdir(os.path.join('DataSet_Razmetra',region)):
        for unknown in os.listdir(os.path.join('DataSet_Razmetra',region,corupt_type)):
            for ed in os.listdir(os.path.join('DataSet_Razmetra',region,corupt_type, unknown)):
                if 'Edition' in ed:
                    path = os.path.join('DataSet_Razmetra',region,corupt_type, unknown,ed)
                    if os.path.exists(os.path.join(path,'Edition_Text.docx')):
                        try:
                            doc = os.path.join(path,'Edition_Text.docx')
                            document = docx.Document(doc)
                            for paragraph in document.paragraphs:
                                highlight = ""
                                for run in paragraph.runs:
                                    if run.font.color.rgb == RGBColor(255, 000, 000):
                                        highlight += run.text                                
                                if highlight:
                                    highlight = highlight.replace(u'\xa0', u' ').replace('\n', ' ')
                                    highlights_red[corupt_type].append(highlight)
                        except:
                            problem_red.append(doc)
                    else:
                        doc = os.path.join(path,'Edition_Text.doc')
                        problem_red.append(doc)

In [ ]:
docs_dict = collections.defaultdict(list)
problem_docs = []
for region in os.listdir('DataSet_Razmetra'):
    if region == 'РСО-Алания':
        continue
    for corupt_type in os.listdir(os.path.join('DataSet_Razmetra',region)):
        for unknown in os.listdir(os.path.join('DataSet_Razmetra',region,corupt_type)):
            for ed in os.listdir(os.path.join('DataSet_Razmetra',region,corupt_type, unknown)):
                if 'Edition' in ed:
                    path = os.path.join('DataSet_Razmetra',region,corupt_type, unknown,ed)
                    if os.path.exists(os.path.join(path,'NC_Edition_Text.docx')):
                        try:
                            doc = os.path.join(path,'NC_Edition_Text.docx')
                            document = docx.Document(doc)
                            for paragraph in document.paragraphs:
                                highlight = ""
                                for run in paragraph.runs:
                                    highlight += run.text                                
                                if highlight:
                                    highlight = highlight.replace(u'\xa0', u' ').replace('\n', ' ').replace('\t', ' ')
                                    pure = highlight.lower()
                                    pure = re.sub(r'[^\w\s]','',pure)
                                    words = []
                                    for word in pure.split(' '):
                                        p = morph.parse(word)[0]
                                        words.append(p.normal_form)
                                    pure = [w for w in words if not word in russian_stopwords]
                                    pure = ' '.join(pure)
                                    if pure:
                                        docs_dict[doc].append(pure)
                        except:
                            problem_docs.append(doc)
                    else:
                        doc = os.path.join(path,'Edition_Text.doc')
                        problem_docs.append(doc)
with open('data_texts_cor.pickle', 'wb') as f:
    pickle.dump(docs_dict, f)
    
docs_dict_no = collections.defaultdict(list)
problem_docs_no = []
for region in os.listdir('DataSet_Razmetra'):
    if region == 'РСО-Алания':
        continue
    for corupt_type in os.listdir(os.path.join('DataSet_Razmetra',region)):
        for unknown in os.listdir(os.path.join('DataSet_Razmetra',region,corupt_type)):
            for ed in os.listdir(os.path.join('DataSet_Razmetra',region,corupt_type, unknown)):
                if 'Edition' in ed:
                    path = os.path.join('DataSet_Razmetra',region,corupt_type, unknown,ed)
                    if os.path.exists(os.path.join(path,'NC_Edition_Text.docx')):
                        try:
                            doc = os.path.join(path,'NC_Edition_Text.docx')
                            document = docx.Document(doc)
                            for paragraph in document.paragraphs:
                                highlight = ""
                                for run in paragraph.runs:
                                    highlight += run.text                                
                                if highlight:
                                    highlight = highlight.replace(u'\xa0', u' ').replace('\n', ' ').replace('\t', ' ')
                                    pure = highlight.lower()
                                    pure = re.sub(r'[^\w\s]','',pure)
                                    words = []
                                    for word in pure.split(' '):
                                        p = morph.parse(word)[0]
                                        words.append(p.normal_form)
                                    pure = [w for w in words if not word in russian_stopwords]
                                    pure = ' '.join(pure)
                                    if pure:
                                        docs_dict_no[doc].append(pure)
                        except:
                            problem_docs_no.append(doc)
                    else:
                        doc = os.path.join(path,'Edition_Text.doc')
                        problem_docs.append(doc)
with open('data_texts_no_cor.pickle', 'wb') as f:
    pickle.dump(docs_dict_no, f)

In [6]:
with open('highlights_red.pickle', 'rb') as f:
    highlights_red = pickle.load(f)

In [7]:
with open('data.pickle', 'rb') as f:
    highlights = pickle.load(f)

In [8]:
morph = pymorphy2.MorphAnalyzer()
pure_highlights = {}
for i in highlights.keys():
    tok = []
    for j in highlights[i]:
        if re.search('[а-яА-Я]',j):
            pure = j.replace(u'\xa0', u' ').replace('\n', ' ')
            pure = pure.lower()
            pure = re.sub(r'[^\w\s]','',pure)
            words = []
            for word in pure.split(' '):
                p = morph.parse(word)[0]
                words.append(p.normal_form)
            pure = [w for w in words if not word in russian_stopwords]
            pure = ' '.join(pure)
            if pure:
                tok.append(pure)
        pure_highlights[i] = list(set(tok))
with open('pure_highlights.pickle', 'wb') as f:
    pickle.dump(pure_highlights, f)        

In [9]:
morph = pymorphy2.MorphAnalyzer()
pure_highlights_red = {}
for i in highlights_red.keys():
    tok = []
    for j in highlights_red[i]:
        if re.search('[а-яА-Я]',j):
            pure = j.replace(u'\xa0', u' ').replace('\n', ' ')
            pure = pure.lower()
            pure = re.sub(r'[^\w\s]','',pure)
            words = []
            for word in pure.split(' '):
                p = morph.parse(word)[0]
                words.append(p.normal_form)
            pure = [w for w in words if not word in russian_stopwords]
            pure = ' '.join(pure)
            if pure:
                tok.append(pure)
        pure_highlights_red[i] = list(set(tok))
with open('pure_highlights_red.pickle', 'wb') as f:
    pickle.dump(pure_highlights_red, f)

In [ ]:
docs_dict_label = collections.defaultdict(list)
problem_docs_label = []
label_dict = {'3_1':1,'3_2':2,'3_3':3,'3_4':4,'3_5':5,'3_6':6,'3_7':7,'3_8':8,'3_9':9,'4_1':10,'4_2':11,'4_4':12}
for region in os.listdir('DataSet_Razmetra'):
    if region == 'РСО-Алания':
        continue
    for corupt_type in os.listdir(os.path.join('DataSet_Razmetra',region)):
        for unknown in os.listdir(os.path.join('DataSet_Razmetra',region,corupt_type)):
            for ed in os.listdir(os.path.join('DataSet_Razmetra',region,corupt_type, unknown)):
                if 'Edition' in ed:
                    path = os.path.join('DataSet_Razmetra',region,corupt_type, unknown,ed)
                    if os.path.exists(os.path.join(path,'NC_Edition_Text.docx')):
                        try:
                            doc = os.path.join(path,'NC_Edition_Text.docx')
                            document = docx.Document(doc)
                            docs_dict_label[doc].append(0)
                            docs_dict_label[doc].append(0)
                        except:
                            problem_docs_label.append(doc)
                    if os.path.exists(os.path.join(path,'Edition_Text.docx')):
                        try:
                            doc = os.path.join(path,'Edition_Text.docx')
                            document = docx.Document(doc)
                            docs_dict_label[doc].append(1)
                            docs_dict_label[doc].append(label_dict[corupt_type])
                        except:
                            problem_docs_label.append(doc)
                    else:
                        doc = os.path.join(path,'Edition_Text.doc')
                        problem_docs_label.append(doc)

In [ ]:
corpus = []
for i in docs_dict.values():
    for j in i:
        corpus.append(j)
for i in docs_dict_no.values():
    for j in i:
        corpus.append(j)
for i in highlights.values():
    for j in i:
        corpus.append(j)
for i in higligts_red.values():
    for j in i:
        corpus.append(j)
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
vectorizer.fit(corpus)

In [ ]:
X_train = []
y_train_simple = []
y_train = []
import random
deom scipy.spatial.distance import cosine
for i in sorted(docs_dict.keys()):
    tmp = ''
    for par in docs_dict[i]:
        tmp = tmp + par + ' '
    distance_vec = []
    for cor in sorted(pure_highlights.keys()):
        cor_par = ''
        for form in cor:
            cor_par = cor_par + form + ' '
        sim = cosine(vectorizer.transform([cor_par]), vectorizer.transform([tmp])
        distance_vec.append(sim)
    X_train.append(distance_vec)
    y_train.append(docs_dict_label[i][1])
    y_train_simple(docs_dict_label[i][0])

In [ ]:
import sklearn.linear_model from SGDClassifier
classifier = SGDClassifier()
classifier.fit(X_train,y_train)